In [1]:
import json
from tqdm import tqdm
import torch
from sentence_transformers import util

In [2]:
big = "memoryalpha"
small = "stexpanded"

embeddings = "dogtag_bgem3"
communities = "leiden"

node_sim_weight = 0.9
community_sim_weight = 0.1
# neighbor_sim_weight = 0.5

In [3]:
mapping_file_big = "./_input/mappings/" + big + ".json"
mapping_file_small = "./_input/mappings/" + small + ".json"

communities_big_file = "./_input/communities/" + communities + "/" + big + ".txt"
communitiy_embeddings_big_file = "./_input/community_embeddings/" + communities + "_" + embeddings + "/" + big + ".json"

communities_small_file = "./_input/communities/" + communities + "/" + big + ".txt"
communitiy_embeddings_small_file = "./_input/community_embeddings/" + communities + "_" + embeddings + "/" + big + ".json"

node_embeddings_small_file = "./_input/node_embeddings/" + embeddings + "/" + small + ".json"
url_embeddings_small_file = "./_input/url_embeddings/" + small + ".json"

exact_match_file = "./_input/exact_match/" + big + "-" + small + ".json"
gold_pairs_file = "./_input/gold_pairs/" + big + "-" + small + ".txt"

top10pairs_file = "./_input/top10pairs/" + embeddings + "/" + small + "-" + big + ".json"

In [4]:
big_communities = []
small_communities = []
gold_pairs = []

with open(communities_big_file) as cbf:
    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        big_communities.append(numbers_set)

with open(communities_small_file) as csf:
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        small_communities.append(numbers_set)

with open(gold_pairs_file) as gpf:
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

with open(exact_match_file) as file:
    exact_match = json.load(file)

with open(communitiy_embeddings_big_file) as cebf:
    community_embeddings_big = json.load(cebf)
    community_embeddings_big = {k: list(v.values()) for k, v in community_embeddings_big.items()}

with open(communitiy_embeddings_small_file) as cesf:
    community_embeddings_small = json.load(cesf)
    community_embeddings_small = {k: list(v.values()) for k, v in community_embeddings_small.items()}

with open(top10pairs_file) as file:
    top10pairs = json.load(file)

with open(node_embeddings_small_file) as nesf:
    node_embeddings_small = json.load(nesf)

with open(url_embeddings_small_file) as uesf:
    url_embeddings_small = json.load(uesf)

FileNotFoundError: [Errno 2] No such file or directory: './_input/top10pairs/dogtag_bgem3/stexpanded-memoryalpha.json'

In [19]:
gold_exact = list()
gold_not_exact = list()

for p in gold_pairs:
    if [p[0], p[1]] in exact_match:
        gold_exact.append([p[0], p[1]])
    else:
        gold_not_exact.append([p[0], p[1]])

merged_node_embeddings_small = {key: node_embeddings_small[key] if key in node_embeddings_small else url_embeddings_small[key] for key in set(node_embeddings_small) | set(url_embeddings_small)}
merged_node_embeddings_small = {str(k): merged_node_embeddings_small[str(k)] for k in sorted(map(int, merged_node_embeddings_small.keys()))}

top1dict = dict()

for k, v in top10pairs.items():
    top1dict[int(k)] = int(v[0][0])

In [20]:
node_to_community_embeddings_small = dict()
index = 0
for community in small_communities:
    for node in community:
        node_to_community_embeddings_small[str(node)] = community_embeddings_small[str(index)]
    index += 1

node_to_community_embeddings_big = dict()
index = 0
for community in big_communities:
    for node in community:
        node_to_community_embeddings_big[node] = community_embeddings_big[str(index)]
    index += 1

top1dict_reordered = dict()

for k in tqdm(top10pairs.keys()):

    node_embeds = torch.Tensor(merged_node_embeddings_small[str(k)])
    try:
        node_embeds = torch.Tensor(node_to_community_embeddings_small[str(k)])
    except KeyError:
        pass

    compare_list = [
        node_to_community_embeddings_big.get(int(item[0]))
        for item in top10pairs[str(k)]
        if int(item[0]) in node_to_community_embeddings_big
    ]
    big_torch_embeds = torch.Tensor(compare_list)

    community_order = util.semantic_search(node_embeds, big_torch_embeds)

    best_score = 0

    for item in community_order[0]:
        score = (
                node_sim_weight * top10pairs[str(k)][item['corpus_id']][1] +
                community_sim_weight * item['score']
        )
        id_node = top10pairs[str(k)][item['corpus_id']][0]
        if score > best_score:
            best_score = score
            top1dict_reordered[int(str(k))] = int(id_node)

100%|██████████| 15524/15524 [00:07<00:00, 2134.70it/s]


In [21]:
print("############### SETTINGS ################")
print("From:            " + small)
print("To:              " + big)
print("Embeddings:      " + embeddings)
print("Communities:     " + communities)
print("Node sim weight: " + str(node_sim_weight))
print("Comm sim weight: " + str(community_sim_weight))

print("############ ALL GOLD PAIRS #############")
print("Count:           " + str(len(gold_pairs)))

found = 0
all_pairs = len(gold_pairs)
# for gold_pair in tqdm(gold_pairs):
for gold_pair in gold_pairs:
    if top1dict.get(gold_pair[1]) == gold_pair[0]:
        found += 1
print("Top 1:           " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_pairs)
# for gold_pair in tqdm(gold_pairs):
for gold_pair in gold_pairs:
    if top1dict_reordered.get(gold_pair[1]) == gold_pair[0]:
        found += 1
print("Reordered:       " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

print("############## EXACT MATCH ##############")
print("Count:           " + str(len(gold_exact)))

found = 0
all_pairs = len(gold_exact)
# for gold_pair in tqdm(gold_exact):
for gold_pair in gold_exact:
    if top1dict.get(gold_pair[1]) == gold_pair[0]:
        found += 1
print("Top 1:           " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_exact)
# for gold_pair in tqdm(gold_exact):
for gold_pair in gold_exact:
    if top1dict_reordered.get(gold_pair[1]) == gold_pair[0]:
        found += 1
print("Reordered:       " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

print("############ NOT EXACT MATCH ############")
print("Count:           " + str(len(gold_not_exact)))

found = 0
all_pairs = len(gold_not_exact)
# for gold_pair in tqdm(gold_not_exact):
for gold_pair in gold_not_exact:
    if top1dict.get(gold_pair[1]) == gold_pair[0]:
        found += 1
print("Top 1:           " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

found = 0
all_pairs = len(gold_not_exact)
# for gold_pair in tqdm(gold_not_exact):
for gold_pair in gold_not_exact:
    if top1dict_reordered.get(gold_pair[1]) == gold_pair[0]:
        found += 1
print("Reordered:       " + str(found) + " (" + f"{found / all_pairs * 100:.5f}" + "%)")

############### SETTINGS ################
From:            stexpanded
To:              memoryalpha
Embeddings:      dogtag_bgelarge
Communities:     leiden
Node sim weight: 0.9
Comm sim weight: 0.1
############ ALL GOLD PAIRS #############
Count:           1779
Top 1:           1390 (78.13378%)
Reordered:       1391 (78.18999%)
############## EXACT MATCH ##############
Count:           1631
Top 1:           1294 (79.33783%)
Reordered:       1297 (79.52177%)
############ NOT EXACT MATCH ############
Count:           148
Top 1:           96 (64.86486%)
Reordered:       94 (63.51351%)
